In [1]:
# Agent ID 설정
agent_id = "YOUR_AGENT_ID"  # Agent ID
agent_alias_id = "TSTALIASID"  # Test Agent Alias ID that is created initialization step
session_id = "test-session-1"
region_name = "us-west-2"

In [2]:
import boto3
import json
import time

# Bedrock 클라이언트 생성
bedrock_agent_runtime = boto3.client(
    service_name='bedrock-agent-runtime',
    region_name=region_name
)

In [3]:
def ask_agent(question):
    print(f"\nQ: {question}")
    
    try:
        response = bedrock_agent_runtime.invoke_agent(
            agentId=agent_id,
            agentAliasId=agent_alias_id,
            sessionId=session_id,
            inputText=question
        )
        
        # EventStream 처리
        completion = response['completion']
        
        print("A: ", end='', flush=True)
        # EventStream의 각 이벤트를 처리하며 실시간 출력
        for event in completion:
            if 'chunk' in event:
                chunk_data = event['chunk']['bytes'].decode()
                try:
                    # JSON 형식으로 파싱 시도
                    chunk_json = json.loads(chunk_data)
                    if isinstance(chunk_json, dict) and 'completion' in chunk_json:
                        text = chunk_json['completion']
                    else:
                        text = str(chunk_json)
                except json.JSONDecodeError:
                    text = chunk_data
                
                # 한 글자씩 출력
                for char in text:
                    print(char, end='', flush=True)
                    time.sleep(0.02)  # 출력 속도 조절
        
        print()  # 줄바꿈
    
    except Exception as e:
        print(f"Error: {str(e)}")

In [4]:
# example questions
questions = [
    # "2025년 5월 16일 서울의 날씨가 어땠었는지 알려주세요.",
    "Tell me what the weather was like in Seoul on May 16th, 2025.",
    # "현재 서울의 시간을 24시간 형식으로 알려주세요.",
    "Tell me the current time in Seoul using the format HH:mm:ss."
]

# ask multiple questions
for question in questions:
    answer = ask_agent(question)


Q: Tell me what the weather was like in Seoul on May 16th, 2025.
A: The weather in Seoul on May 16th, 2025, is expected to be as follows:
- Temperature: High of 70°F (21°C) and low of 59°F (15°C)
- Precipitation: 0.307 inches (7.8 mm) with a 70% chance of rain
- Humidity: 77%
- Sunrise: 05:22:08
- Sunset: 19:35:13

This information is based on the <REDACTED> results from DuckDuckGo.


Q: Tell me the current time in Seoul using the format HH:mm:ss.
A: The current time in Seoul is 13:38:21.
